In [1]:
#load "Paket.fsx"
Paket.Version [ ("Alea", "3.1.0-beta1") ]
Paket.Package [ "NUnit" ]

<null>

In [2]:
#load "packages/Alea/Alea.fsx"
#r "packages/Alea/lib/net45/Alea.Parallel.dll"
#r "packages/NUnit/lib/net45/nunit.framework.dll"

In [3]:
open System
open Alea
open Alea.CSharp
open Alea.Parallel

let gpu = Gpu.Default
gpu

[0|6.0|Tesla P100-PCIE-16GB]

In [25]:
let calcPI_1 verbose rng seed batchSize batchs =
    let t = System.Diagnostics.Stopwatch.StartNew()
    use session = new Session(gpu)
    session.SetRandomType(rng)
    session.SetRandomSeed(seed)
    let points = session.Allocate<double2>(batchSize)
    let values = session.Allocate<float>(batchSize)
    let pis = session.Allocate<float>(batchs)

    for iBatch = 1 to batchs do
        // generate random numbers
        let offset = (uint64 batchSize) * (uint64 iBatch)
        session.RandomUniform(points, offset)
        
        session.For(0, batchSize, (fun i ->
            let point = points.[i]
            let d = point.x * point.x + point.y * point.y
            values.[i] <- if d < 1.0 then 4.0 else 0.0))

        session.Aggregate(batchSize,
                         (fun i -> values.[i]),
                         (fun value -> pis.[iBatch - 1] <- value / (float batchSize)),
                         (fun a b -> a + b))

    let pis = Gpu.CopyToHost(pis)
    let pi = pis |> Array.average
    t.Stop()

    if verbose then
        printfn "%A" pis
        printfn "PI = %f, %A" pi t.Elapsed

    pi, t.Elapsed, ((pis |> Array.distinct).Length)

In [26]:
let calcPI_2 verbose rng seed batchSize batchs =
    let t = System.Diagnostics.Stopwatch.StartNew()
    use session = new Session(gpu)
    session.SetRandomType(rng)
    session.SetRandomSeed(seed)
    let points = session.Allocate<double2>(batchSize)
    let pis = session.Allocate<float>(batchs)

    for iBatch = 1 to batchs do
        // generate random numbers
        let offset = (uint64 batchSize) * (uint64 iBatch)
        session.RandomUniform(points, offset)

        session.Aggregate(batchSize,
                         (fun i ->
                            let point = points.[i]
                            let d = point.x * point.x + point.y * point.y
                            if d < 1.0 then 4.0 else 0.0),
                         (fun value -> pis.[iBatch - 1] <- value / (float batchSize)),
                         (fun a b -> a + b))

    let pis = Gpu.CopyToHost(pis)
    let pi = pis |> Array.average
    t.Stop()

    if verbose then
        printfn "%A" pis
        printfn "PI = %f, %A" pi t.Elapsed

    pi, t.Elapsed, ((pis |> Array.distinct).Length)


In [31]:
let calcPI rng =
    let batchs = 50
    let batchSize = 10000000
    let seed = 1UL
    let pi1, t1, l1 = calcPI_1 false rng seed batchSize batchs
    let pi2, t2, l2 = calcPI_2 false rng seed batchSize batchs
    sprintf "%.8f [%A|%d] %.8f [%A|%d]" pi1 t1 l1 pi2 t2 l2

In [32]:
calcPI cuRAND.RngType.PSEUDO_DEFAULT

"3.14161200 [00:00:08.3936840|50] 3.14161200 [00:00:08.4166988|50]"

In [33]:
calcPI cuRAND.RngType.PSEUDO_DEFAULT

"3.14161200 [00:00:08.3430636|50] 3.14161200 [00:00:08.4902345|50]"

In [34]:
calcPI cuRAND.RngType.PSEUDO_MRG32K3A

"3.14167326 [00:00:02.8268253|50] 3.14167326 [00:00:03.2572815|50]"

In [35]:
calcPI cuRAND.RngType.PSEUDO_PHILOX4_32_10

"3.14147120 [00:00:00.9797533|1] 3.14147120 [00:00:01.4185882|1]"